<a href="https://colab.research.google.com/github/MichalWlodkowski/US_speeches/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scrapy import Selector
import requests
import numpy as np
import pandas as pd
import re

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

In [0]:
us_all = 'https://en.wikisource.org/wiki/Portal:Inaugural_Speeches_by_United_States_Presidents'

In [0]:
html = requests.get(us_all).content
sel = Selector( text = html )
xpath3 = '//@href'
d = pd.DataFrame(sel.xpath(xpath3).extract())
speech = d[d[0].str.contains('Inaugural_Address')].reset_index(drop = True)
speech.columns = ['links']

In [6]:
def frag(x):
    wh = x[6:x.find('%')].replace('_',' ')
    return wh
speech['president'] = speech['links'].apply(frag)
speech.head()
#speech['links'].apply(

,links,president
0,/wiki/George_Washington%27s_First_Inaugural_Ad...,George Washington
1,/wiki/George_Washington%27s_Second_Inaugural_A...,George Washington
2,/wiki/John_Adams%27_Inaugural_Address,John Adams
3,/wiki/Thomas_Jefferson%27s_First_Inaugural_Add...,Thomas Jefferson
4,/wiki/Thomas_Jefferson%27s_Second_Inaugural_Ad...,Thomas Jefferson


In [7]:
speech['full_links'] = speech['links'].apply(lambda x: 'https://en.wikisource.org' + x)
speech.head(1)

,links,president,full_links
0,/wiki/George_Washington%27s_First_Inaugural_Ad...,George Washington,https://en.wikisource.org/wiki/George_Washingt...


In [0]:
plain_text = []
text_lines = []

#for each in range(len(speech)-3,len(speech)):
for each in range(len(speech)):
    
    html = requests.get(speech.full_links.loc[each]).content
    sel = Selector (text = html)
    xpath3 = '//*[@class="mw-parser-output"]//p//text()'
    d = sel.xpath(xpath3).extract()
    e = ''.join(d)
    e = e.replace('\nThis work is in the public domain in the United States because it is a work of the United States federal government (see 17\xa0U.S.C.\xa0105).\n','')
    f = e.replace('\n',' ')
    plain_text.append(f)
    text_lines.append(e)    

In [0]:
def hm_words(x):
    words = len(x.split())
    return words

def hm_ch_words(x):
    words_ch = round(np.mean([len(x) for x in x.split()]),2)
    return words_ch

def n_sentences(x):
    sent = len(x.split('.'))
    return sent

def n_parag(x):
    par =  len(x.split('\n'))
    return par

def word_in_sent(x):
    sent_w = round(np.mean([len(x.split()) for x in x.split('.')]),1)
    return sent_wm

In [12]:
speeches = pd.DataFrame(plain_text, columns = ['speeches'])
speeches['line_text'] = pd.DataFrame(text_lines)
speeches['characters'] = speeches.speeches.apply(len)
speeches['words'] = speeches.speeches.apply(hm_words)
speeches['ch_in_words'] = speeches.speeches.apply(hm_ch_words)
speeches['sentences'] = speeches.speeches.apply(n_sentences)
speeches['paragraphs'] = speeches.line_text.apply(n_parag)

#speeches['words_in_sent'] = speeches.speeches.apply(word_in_sent)
speeches['ch_in_sent'] = round(speeches['characters'] / speeches['sentences'],1)
speeches.tail(2)

,speeches,line_text,characters,words,ch_in_words,sentences,paragraphs,ch_in_sent
56,"Vice President Biden, Mr. Chief Justice, Memb...","\nVice President Biden, Mr. Chief Justice, Mem...",11954,2114,4.65,86,23,139.0
57,"Chief Justice Roberts, President Carter, Presi...","Chief Justice Roberts, President Carter, Presi...",8413,1456,4.78,96,16,87.6
